In [ ]:
import pandas as pd
import json
import glob
import os
import numpy
from scipy.optimize import linprog
import numpy as np
import tqdm

import matplotlib.pylab as plt

In [ ]:
def load_data(path):

    res = {}
    for f in glob.glob(path):
        with open(f, "r") as fh:
            res[os.path.basename(f).replace(".json", "")] = pd.DataFrame(json.load(fh))

    df_prod = res["production_schedule"]
    df_prod["cu_pct"] = df_prod["chemistry"].apply(lambda x: x['cu_pct'])
    df_prod.drop("chemistry", axis=1, inplace=True)

    df_chem = res["previous_heats_with_properties"]
    df_chem["cu_pct"] = df_chem["chemistry"].apply(lambda x: x['cu_pct'])
    for b in ["bushling", "pig_iron", 'municipal_shred', 'skulls']:
        df_chem[b] = df_chem["actual_recipe"].apply(lambda x: x[b])
    
    df_chem["total_input"] = df_chem[["bushling", "pig_iron", 'municipal_shred', 'skulls']].sum(axis=1)
    for b in ["bushling", "pig_iron", 'municipal_shred', 'skulls']:
        df_chem[b] /= df_chem["total_input"]
        
    df_chem["yield"] = df_chem["tap_weight"] / df_chem["total_input"]
    
    df_chem.drop("chemistry", axis=1, inplace=True)
    df_chem.drop("actual_recipe", axis=1, inplace=True)

    df_order = res['scrap_orders']

    df_constr = res['constraints']
    df_constr = pd.DataFrame(map(lambda x: x[0], df_constr.values.tolist()))
    
    df_inven = res['scrap_inventory']
    
    return df_prod, df_chem, df_order, df_constr, df_inven

In [ ]:
df_prod, df_chem, df_order, df_constr, df_inven = load_data("data/1/*.json")
df_prod

In [ ]:
df_chem

# scrap quality estimation

In [ ]:
y = df_chem["cu_pct"].iloc[:4, ].values
X = df_chem[["bushling", "pig_iron", "municipal_shred", "skulls"]].iloc[:4, ].values

In [ ]:
# a = numpy.linalg.solve(X, y)

In [ ]:
def get_cost(df_order, df_inv, commodity):
    df = df_order.sort_values("order_date", ascending=False)
    df = df[df["scrap_type"] == commodity]
    return df

In [ ]:
get_cost(df_order, df_inven, "pig_iron")

In [ ]:
df_order.sort_values("order_date", ascending=False)

In [ ]:
df_constr

In [ ]:
df_inven

In [ ]:
def generate_schedule(commodities=[{"name": "A", "price": 10, "cu": 0.01, "inventory": 1000, "yield": 0.99}, 
                                   {"name": "B", "price": 20, "cu": 0.05, "inventory": 1000, "yield": 0.95}],
                     schedule=[{"cu": 0.01, "weight": 200}, 
                               {"cu": 0.05, "weight": 400},
                              {"cu": 0.10, "weight": 500}]):
    
    p = [c["price"] for c in commodities] * len(schedule)

    print("prices")
    print(p)

    A_lower = np.repeat([1/s["weight"] for s in schedule], len(commodities)).reshape([-1, len(commodities)]) * np.array([c["cu"] for c in commodities] * len(schedule)).reshape([-1, 2])
    b_lower = [s["cu"] for s in schedule]

    print("A_lower")
    print(A_lower)

    A_equal = np.array([c["cu"] for c in commodities * len(schedule)]).reshape(-1, 2) / np.array([s["weight"] for s in schedule]).reshape([1, -1]).transpose()
    b_equal = np.array([s["cu"] for s in schedule]).reshape([1, -1]).transpose()

    print("A_equal")
    print(A_equal)

    lb = [0, 0]
    lu = [c["inventory"] for c in commodities]

    res = linprog(c=p, A_ub=A_lower, b_ub=b_lower, bounds=[lb, lu])

    print(res)
    
generate_schedule()
    
    
    
    

In [ ]:
def generate_schedule_random(
        commodities=[{"name": "A", "price": 10, "cu": 0.01, "inventory": 1000, "yield": 0.99},
                     {"name": "B", "price": 20, "cu": 0.05, "inventory": 1000, "yield": 0.95}],
        schedule=[{"cu": 0.02, "weight": 200},
                  {"cu": 0.05, "weight": 400},
                  {"cu": 0.10, "weight": 500}],
        constraints=[{"A": {"min": 10, "max": 200}},
                     {"B": {"min": 5}}],
        iterations=10000):
    
    df = pd.DataFrame([{"name": "A", "min": 10, "max": 300}, {"name": "B", "min": 5}])
    
    df = df.set_index("name")
    df = df.reindex([c["name"] for c in commodities])
    df["min"] = df["min"].fillna(0)
    df["max"] = df["max"].fillna(np.inf)
    
    r_min = df["min"].values
    r_max = df["max"].values
    
    print(r_min)
    print(r_max)
    
    
    c_cu = np.array(np.array([c["cu"] for c in commodities]))
    s_cu = np.array(np.array([s["cu"] for s in schedule]))
    s_weight = np.array([s["weight"] for s in schedule])
    c_inv = np.array([c["inventory"] for c in commodities])
    c_price = np.array([c["price"] for c in commodities])

    # print(c_cu)
    # print(s_weight)
    # print("start")

    # todo: consider yield
    # todo: consider constraints
    cost = [np.inf]
    cu_predicted = None
    res = None
    miss_chem, miss_inven, miss_constr = 0, 0, 0
    valid = 0

    for _ in tqdm.tqdm(range(iterations)):
        
        bad = False
        
        # generate schedule
        res_t = []
        for s in [s["weight"] for s in schedule]:
            res_t.append(np.random.random_integers(0, s, size=len(commodities)))
        res_t = np.vstack(res_t)

        # make weights equal to schedule weights by fixing last commodity
        res_t[:, -1] = s_weight - res_t[:, :-1].sum(axis=1)

        # check chemistry
        cu_predicted_t = res_t.dot(c_cu.transpose()) / s_weight
        if not (cu_predicted_t <= s_cu).all():
            miss_chem += 1
            bad = True

        # check inventory
        used = res_t.sum(axis=0)
        if not (used <= c_inv).all():
            miss_inven += 1
            bad = True

        #check side constraints
        if (res_t < r_min).any() or (res_t > r_max).any():
            miss_constr += 1
            bad = True
        
        if bad:
            cost += [cost[-1]]
            continue
        
        valid += 1
        cost_temp = np.sum(used * c_price)

        # check if costs are lower
        if cost_temp < cost[-1]:
            res = res_t
            cu_predicted = cu_predicted_t
            cost += [cost_temp]
    
    plt.plot(cost[1:])
    stats = pd.Series({"iterations": 10000, 
                       "miss_chem": miss_chem, "miss_inven": miss_inven, 
                       "miss_constr": miss_constr, "valid": valid, 
                       "cost": cost[-1], "predicted_cu": cu_predicted})
    return res, stats

In [ ]:
generate_schedule_random()

In [ ]:
pd.DataFrame([{"name": "A", "min": 10, "max": 100}, 
              {"name": "B", "min": 5}])

In [ ]:
df = pd.DataFrame([{"name": "A", "min": 10, "max": 100}, 
                              {"name": "B", "min": 5}])